Create Model and Flowsheet
------------------------------------------

In [ ]:
from pyomo.environ import ConcreteModel, SolverFactory, SolverStatus, TerminationCondition
from idaes.core import FlowsheetBlock

In [ ]:
m = ConcreteModel()

In [ ]:
m.fs = FlowsheetBlock(default={"dynamic": False})

Define Properties
--------------------------

In [ ]:
import idaes.property_models.ideal.BTX_ideal_VLE as ideal_props

In [ ]:
m.fs.properties = ideal_props.BTXParameterBlock()

Adding Flash Unit
--------------------------

In [ ]:
from idaes.unit_models import Flash

In [ ]:
m.fs.flash = Flash(default={"property_package": m.fs.properties})

Set Operating Conditions
-------------------------------------

In [ ]:
from idaes.ui.report import degrees_of_freedom
print("Degrees of Freedom =", degrees_of_freedom(m))

In [ ]:
m.fs.flash.inlet.flow_mol.fix(1)
m.fs.flash.inlet.temperature.fix(368)
m.fs.flash.inlet.pressure.fix(101325)
m.fs.flash.inlet.mole_frac[0, "benzene"].fix(0.5)
m.fs.flash.inlet.mole_frac[0, "toluene"].fix(0.5)

In [ ]:
m.fs.flash.heat_duty.fix(0)
m.fs.flash.deltaP.fix(0)

In [ ]:
print("Degrees of Freedom =", degrees_of_freedom(m))

Initializing the Model
------------------------------

In [ ]:
m.fs.flash.initialize()

In [ ]:
solver = SolverFactory('ipopt')

In [ ]:
results = solver.solve(m, tee=True)

Viewing the Results
-----------------------------

In [ ]:
m.fs.flash.vap_outlet.display()

In [ ]:
m.fs.flash.liq_outlet.display()

Studying Purity as a Function of Heat Duty
--------------------------------------------------------------

In [ ]:
m.fs.flash.heat_duty.fix(-17000)

q = []
vap_frac = []
y_benzene = []
recovery = []

while m.fs.flash.heat_duty[0].value <= 25000:
    print("Q = ", m.fs.flash.heat_duty[0].value)
    results = solver.solve(m)
    
    if (results.solver.termination_condition != TerminationCondition.optimal or
            results.solver.status != SolverStatus.ok):
        print("Solver error!")
        break
    
    q.append(m.fs.flash.heat_duty[0].value)
    vap_frac.append(m.fs.flash.vap_outlet.flow_mol[0].expr.value)
    y_benzene.append(m.fs.flash.vap_outlet.mole_frac[0, "benzene"].expr.value)
    recovery.append(100*m.fs.flash.vap_outlet.flow_mol[0].expr.value*m.fs.flash.vap_outlet.mole_frac[0, "benzene"].expr.value/0.5)
    
    m.fs.flash.heat_duty.fix(m.fs.flash.heat_duty[0].value + 200)

In [ ]:
from matplotlib import pyplot

pyplot.figure("Vapor Fraction")
pyplot.plot(q, vap_frac)
pyplot.grid()
pyplot.xlabel("Heat Duty [J]")
pyplot.ylabel("Vapor Fraction [-]")

In [ ]:
pyplot.figure("Purity")
pyplot.plot(q, y_benzene)
pyplot.grid()
pyplot.xlabel("Heat Duty [J]")
pyplot.ylabel("Vapor Benzene Mole Fraction [-]")

In [ ]:
pyplot.figure("Recovery")
pyplot.plot(q, recovery)
pyplot.grid()
pyplot.xlabel("Heat Duty [J]")
pyplot.ylabel("Benzene Recovery [%]")

In [ ]:
pyplot.figure("Recovery vs. Purity")
pyplot.plot(y_benzene, recovery)
pyplot.grid()
pyplot.xlabel("Vapor Benzene Mole Fraction [-]")
pyplot.ylabel("Benzene Recovery [%]")